In [959]:
import numpy as np
import random
from tqdm.auto import tqdm
from dataclasses import dataclass
import copy
from scipy import stats
from icecream import ic


In [960]:
problem = np.load('../data/problem_0.npz')
x = problem['x']
# x = np.array([[1,1,1], [1,1,1]])

y = problem['y']


# convert normal python array into numpy ndarray
# x = np.array(x)

PROBLEM_SIZE  = np.shape(x)[0]
print(PROBLEM_SIZE)

print(np.shape(x))

2
(2, 1000)


In [961]:
BINARY_OPERATORS = {
    "+": np.add,
    "-": np.subtract,
    "*": np.multiply,
    "/": np.divide
}

#https://numpy.org/doc/2.1/reference/routines.math.html
UNARY_OPERATORS = {
        "": lambda x: x,  
        "sin": np.sin,
        "cos": np.cos,
        "tan":np.tan,
        "log": np.log,
        "arccos": np.arccos,
        "arcsin":np.arcsin,
        "arctan":np.arctan,
        "sqrt":np.sqrt,
        "cbrt":np.cbrt,
        "abs":np.abs,
        "reciprocal":np.reciprocal,
        # "exp": np.exp
    }

#VARIABLES = [f"X_{i}" for i in range(PROBLEM_SIZE)]

#VARIABLES_WEIGHTS = [[1/len(VARIABLES) for _ in range(len(VARIABLES))]]
VARIABLES_MAP = {f"X_{i}": x[i] for i in range(PROBLEM_SIZE)}    # {'X_0': [1, 2, 3], 'X_1': [4, 5, 6], 'X_2': [7, 8, 9]}
#LEAVES = [i for i in range(-MAX_COEFFICIENT, MAX_COEFFICIENT)] + list(VARIABLES_MAP.keys())
# print(VARIABLES_MAP)
# print(LEAVES)



In [962]:
def mse(y_computed: np.ndarray, y_expected: np.ndarray):
    # 100*np.square(y_train-d3584.f(x_train)).sum()/len(y_train):g}")
    return 100 * np.square(y_expected - y_computed).sum() / len(y_expected)

def mse2(y_computed: np.ndarray, y_expected: np.ndarray):
    return np.square(y_expected - y_computed).sum() / len(y_expected)


In [963]:
def optimize_initial_coefficients(x, y):
    # Get statistics from y
    y_mean = np.mean(y)
    y_std = np.std(y)
    y_min, y_max = np.min(y), np.max(y)
    
    # For each feature in x
    coefficients = []
    for i in range(x.shape[0]):
        x_i = x[i]
        x_mean = np.mean(x_i)
        x_std = np.std(x_i)
        
        # Basic scaling coefficient to match ranges
        coeff = y_std / (x_std + 1e-8)  # avoid division by zero
        
        coefficients.append(coeff)
    
    return np.array(coefficients)

def get_coefficient_ranges():
    # Get rough scaling factors
    base_coeffs = optimize_initial_coefficients(x, y)
    
    # Create ranges around these coefficients
    ranges = []
    for coeff in base_coeffs:
        # Create range from 0.1× to 10× the optimized coefficient
        min_coeff = coeff * 0.1
        max_coeff = coeff * 10
        ranges.append((min_coeff, max_coeff))
        
    return ranges

In [964]:
ranges = get_coefficient_ranges()
ranges

[(np.float64(0.10015993129077867), np.float64(10.015993129077867)),
 (np.float64(0.32265347886198437), np.float64(32.265347886198434))]

In [965]:
def compute_coefficient(x_i):
    """returns a random float coefficient between -MAX_COEFFICIENT and MAX_COEFFICIENT"""
    x_array = list(VARIABLES_MAP.keys())
    return np.random.choice(ranges[x_array.index(x_i)])

def optimize_initial_coefficients(x, y):
    # Get statistics from y
    y_mean = np.mean(y)
    y_std = np.std(y)
    y_min, y_max = np.min(y), np.max(y)
    
    # For each feature in x
    coefficients = []
    for i in range(x.shape[0]):
        x_i = x[i]
        x_mean = np.mean(x_i)
        x_std = np.std(x_i)
        
        # Basic scaling coefficient to match ranges
        coeff = y_std / (x_std + 1e-8)  # avoid division by zero
        
        coefficients.append(coeff)
    
    return np.array(coefficients)

def get_constant_ranges(y):
    y_mean = np.mean(y)
    y_std = np.std(y)
    y_min, y_max = np.min(y), np.max(y)
    
    # Create different ranges based on operation context
    ranges = {
        'add_sub': (-y_std, y_std),  # for addition/subtraction
        'mult_div': (-2.0, 2.0),     # for multiplication/division
        'small': (-1.0, 1.0),        # for fine-tuning
        'powers': range(-3, 3)        # for exponents, usually small integers
    }
    return ranges

def generate_constant(operation_type):
    if operation_type in UNARY_OPERATORS.keys():
        return generate_safe_constant() # TODO generate constant generator for unary oeprators
    
    ranges = get_constant_ranges(y)

    
    if random.random() < 0.5:
        operation_type = 'small'
    else:
        if operation_type == '+' or operation_type == '-':
            operation_type = 'add_sub'
        elif operation_type == '*' or operation_type == '/':
            operation_type = 'mult_div'
        
        else:      # Integer constants for exponents
            return np.random.choice(ranges['powers'])
        
    min_val, max_val = ranges[operation_type]
    # Float constants for other operations
    return np.random.uniform(min_val, max_val)



def generate_safe_constant():
    """Generate a constant that would be safe for any operation"""
    y_std = np.std(y)
    y_mean = np.mean(y)
    
    # Start with a conservative range
    min_val = 0.1  # Avoid zero for division
    max_val = min(2.0, y_std)  # Keep it reasonable for both mult and add
    
    return np.random.uniform(min_val, max_val)

In [966]:
# MAX_COEFFICIENT = np.abs(min(y)) # adjustable depending on output scale
# MAX_COEFFICIENT = 1 # adjustable depending on output scale

# def compute_coefficient(x_i):
#     """returns a random float coefficient between -MAX_COEFFICIENT and MAX_COEFFICIENT"""
#     x_array = list(VARIABLES_MAP.keys())
#     return np.random.choice(ranges[x_array.index(x_i)])

def optimize_initial_coefficients(x, y):
    # Get statistics from y
    y_mean = np.mean(y)
    y_std = np.std(y)
    y_min, y_max = np.min(y), np.max(y)
    
    # For each feature in x
    coefficients = []
    for i in range(x.shape[0]):
        x_i = x[i]
        x_mean = np.mean(x_i)
        x_std = np.std(x_i)
        
        # Basic scaling coefficient to match ranges
        coeff = y_std / (x_std + 1e-8)  # avoid division by zero
        
        coefficients.append(coeff)
    
    return np.array(coefficients)

def compute_coefficients():
    # Get rough scaling factors
    base_coeffs = optimize_initial_coefficients(x, y)
    
    # Create ranges around these coefficients
    ranges = []
    for coeff in base_coeffs:
        # Create range from 0.1× to 10× the optimized coefficient
        min_coeff = coeff * 0.1
        max_coeff = coeff * 10
        ranges.append((min_coeff, max_coeff))
        
    return ranges


def are_compatible(operator, value, base=0):
    match operator:
        case "/":
            if not isinstance(value,np.ndarray):
                return False if value == 0 else True
            else : # check if all values are non-zero
                return False if (0 in value) else True
            
        case "^":
            if not isinstance(value, np.ndarray) and not isinstance(base, np.ndarray): # (non array) ^ (non array)
                return False if (base < 0 and not int(value) == value) or (base == 0 and value < 0) else True
            elif not isinstance(value, np.ndarray) and isinstance(base, np.ndarray): # array ^ (non array)
                return False if (not int(value) == value and np.any(base < 0)) or (value < 0 and np.any(base == 0)) else True
            elif isinstance(value, np.ndarray) and not isinstance(base, np.ndarray): # (non array) ^ array
                return False if (base < 0 and any((not int(i) == i) for i in value)) or (base == 0 and np.any(value < 0)) else True
            else : # array ^ array
                for i in range(len(value)):                    
                    if (base[i] < 0 and not int(value[i]) == value[i]) or (base[i] == 0 and value[i] < 0):
                        return False 
                return True
            
        case "log" :
            if not isinstance(value,np.ndarray):
                return False if value <= 0 else True
            else : # check if all values are non-negative
                return False if (np.any(value <= 0)) else True
        
        case "arccos" :
            if not isinstance(value,np.ndarray):
                return False if value < -1 or value > 1 else True
            else : # check if all values are between -1 and 1
                return False if (np.any(value < -1 ) or np.any(value > 1) ) else True
            
        case "arcsin" :
            if not isinstance(value,np.ndarray):
                return False if value < -1 or value > 1 else True
            else : # check if all values are between -1 and 1
                return False if (np.any(value < -1 )or np.any(value > 1) )else True
        
        case "sqrt" :
            if not isinstance(value,np.ndarray):
                return False if value < 0 else True
            else : # check if all values are non-negative
                return False if (np.any(value < 0) )else True

        case "reciprocal" :
            if not isinstance(value,np.ndarray):
                return False if value == 0 else True
            else:
                return False if (np.any(value == 0)) else True
            
        case "tan" :
            if not isinstance(value,np.ndarray):
                k = (value - np.pi / 2) / np.pi
                return False if k.is_integer() else True
            else:
                for i in range(len(value)):
                    k = (value[i] - np.pi / 2) / np.pi
                    if k.is_integer():
                        return False
                return True

        case _:
            return True


In [967]:
# def similarity(operator, x_i, y):
#     return 1 / (1+ mse2(operator(x_i), y))

def similarity(operator, x_i, y):
    # Add normalization to make comparison fairer across different scales
    y_pred = operator(x_i)
    y_norm = (y - y.mean()) / (y.std() + 1e-8)
    y_pred_norm = (y_pred - y_pred.mean()) / (y_pred.std() + 1e-8)
    return 1 / (1 + mse2(y_pred_norm, y_norm))

# def compute_weights_corr():
#     weights = {op: 0 for op in UNARY_OPERATORS.keys()}
    
#     for op in weights:
#         # max_sim = 0
#         max_correlation = 0
#         for i in range(np.shape(x)[0]):
#             if (are_compatible(op, x[i])):
#                 # sim = similarity(UNARY_OPERATORS[op], x[i], y)
#                 # max_sim = max(max_sim, sim)
#                 transformed = UNARY_OPERATORS[op](x[i])
#                 correlation = abs(stats.pearsonr(transformed, y)[0])
#                 max_correlation = max(max_correlation, correlation)
#         # weights[op] = max_sim
#         weights[op] = max_correlation
    
#       # Normalize weights
#     total = sum(weights.values())
#     if total > 0:
#         weights = {k: v/total for k, v in weights.items()}
#     else:
#         # If all correlations failed, use uniform weights
#         weights = {k: 1.0/len(UNARY_OPERATORS) for k in UNARY_OPERATORS.keys()}
        
#     return weights

def compute_weights_sim(operators):
    weights = {op: 0 for op in operators}
    
    for op in weights:
        max_sim = 0
        for i in range(np.shape(x)[0]):
            if (are_compatible(op, x[i])):
                sim = similarity(UNARY_OPERATORS[op], x[i], y)
                max_sim = max(max_sim, sim)
               
        weights[op] = max_sim
    
      # Normalize weights
    total = sum(weights.values())
    if total > 0:
        weights = {k: v/total for k, v in weights.items()}
    else:
        # If all correlations failed, use uniform weights
        weights = {k: 1.0/len(UNARY_OPERATORS) for k in UNARY_OPERATORS.keys()}
        
    return weights

w =  compute_weights_sim(UNARY_OPERATORS.keys())
print(w)
print(sum(w.values()))



{'': np.float64(0.17827438485659855), 'sin': np.float64(0.12257107827692593), 'cos': np.float64(0.060135028009058455), 'tan': np.float64(0.06158121715886166), 'log': np.float64(0.0), 'arccos': np.float64(0.057759902730593476), 'arcsin': np.float64(0.061586571027254), 'arctan': np.float64(0.17087986128844349), 'sqrt': np.float64(0.0), 'cbrt': np.float64(0.16454195867014046), 'abs': np.float64(0.06082183578409751), 'reciprocal': np.float64(0.06184816219802644)}
0.9999999999999999


In [968]:
# # weights
# UNARY_OPERATORS_WEIGHTS = compute_weights_sim()
# BINARY_OPERATORS_WEIGHTS = [[1/len(BINARY_OPERATORS.keys()) for _ in range(len(BINARY_OPERATORS.keys()))]]
# # UNARY_OPERATORS_WEIGHTS

## Printing functions

In [969]:
def print_tree(node, is_root: bool = True):
    """
    Prints the symbolic representation of the tree with the names of the variables
    
    Args:
        node: TreeNode
        is_root: bool
    """
    if not node:
        return

    # Add parentheses around subexpressions unless it's the root
    if not is_root:
        print("(", end="")

    # Traverse the left child
    if node.left:
        print_tree(node.left, is_root=False)

    # Print the current node's value
    print(f"{node.coefficient} {node.value}" if node.coefficient and node.value in VARIABLES_MAP else node.value, end=" ")

    # Traverse the right child
    if node.right:
        print_tree(node.right, is_root=False)

    # Close parentheses if not the root
    if not is_root:
        print(")", end="") 


def print_tree_values(node, is_root: bool = True):
    """ 
    Prints the symbolic representation of the tree with the values of the variables
    
    Args:
        node: TreeNode
        is_root: bool
    """
    if not node:    
        return

    # Add parentheses around subexpressions unless it's the root
    if not is_root:
        print("(", end="")

    # Traverse the left child
    if node.left:
        print_tree_values(node.left, is_root=False)

    # Print the current node's value
    print(f"{node.coefficient} * {VARIABLES_MAP[node.value]}" if node.coefficient and node.value in VARIABLES_MAP else node.value, end=" ")

    # Traverse the right child
    if node.right:
        print_tree_values(node.right, is_root=False)

    # Close parentheses if not the root
    if not is_root:
        print(")", end="") 


def print_expr(node):
    """
    Prints the symbolic representation of the tree with the names of the variables inside an expression
    """
    print_tree(node) 
    print(" = y")

def print_expr_values(node):
    """
    Prints the symbolic representation of the tree with the values of the variables inside an expression
    """
    print_tree_values(node) 
    print(" = ", end="")
    print(node.evaluate_tree_from_node())

## Tree structure


In [989]:
class TreeNode:
    def __init__(self, value):
        self.value = value      # This can be an operator or operand
        self.left = None        # Left child
        self.right = None       # Right child
        self.coefficient = None # multiplicative coefficient for a variable
        # self.depth = 1  # Leaf nodes have depth 1
        # self._update_depth()

    def __copy__(self):
        return TreeNode(self.value, self.left, self.right, self.coefficient, self.depth)

    def __eq__(self, other):
        if not isinstance(other, TreeNode):
            return False
        return self.value == other.value
    
    def _update_depth(self):
        """Update the depth of this node based on its children"""
        left_depth = self.left.depth if self.left else 0
        right_depth = self.right.depth if self.right else 0
        self.depth = max(left_depth, right_depth) + 1
        
    def update_depths_recursive(self):
        """Update depths for this node and all ancestors"""
        if self.left:
            self.left.update_depths_recursive()
        if self.right:
            self.right.update_depths_recursive()
        self._update_depth()

    def get_nodes_from_node(self):
        """ 
        Returns a list of all nodes in the tree starting from the given node
        """
        nodes = []
        if self:
            nodes.append(self)
        if self.left:
            nodes.extend(self.left.get_nodes_from_node())
        if self.right:
            nodes.extend(self.right.get_nodes_from_node())
        return nodes
    
    def get_non_leaves_nodes_from_node(self):
        """
        Returns a list of all non leaf nodes in the tree starting from the given node
        """
        nodes = []
        if self and (self.left or self.right):
            nodes.append(self)
        if self.left:
            nodes.extend(self.left.get_non_leaves_nodes_from_node())
        if self.right:
            nodes.extend(self.right.get_non_leaves_nodes_from_node())
        return nodes
    
    def get_leaves_nodes_from_node(self):
        """
        Returns a list of all leaf nodes in the tree starting from the given node
        """
        nodes = []
        if self and not (self.left or self.right):
            nodes.append(self)
        if self.left:
            nodes.extend(self.left.get_leaves_nodes_from_node())
        if self.right:
            nodes.extend(self.right.get_leaves_nodes_from_node())
        return nodes
        
    def validate_tree_from_node(self):
        """
        Returns True if the tree is syntactically correct without checking domain constraints of operators, False otherwise
        """
        if not self:
            return True
        
        if self.value in BINARY_OPERATORS:
            if not self.left or not self.right:
                return False  # Operators must have two children
            return self.left.validate_tree_from_node() and self.right.validate_tree_from_node()
        
        elif self.value in UNARY_OPERATORS:  # Allow unary operators
            if self.right and not self.left:
                return self.right.validate_tree_from_node()
            return False  # Unary operators must have one child on the right
        
        # elif self.value in VARIABLES_MAP and isinstance(self.value, str):  # Allow variables
        elif self.value in list(VARIABLES_MAP.keys()): # or (self.value > -MAX_COEFFICIENT and self.value < MAX_COEFFICIENT):
            return True
        elif isinstance(self.value, float):
            return True
        else:
            return False  # Invalid value
    
    # (3 + 2) * (4 + 5)        Treenode (value = *, left = Treenode (value = +, left = 3, right = 2), right = Treenode (value = +, left = 4, right = 5))
    def evaluate_tree_from_node(self):
        """
        Returns the value of the expression represented by the tree starting form a specific node
        """
        if not self:
            raise ValueError("Cannot evaluate an empty tree.")
        
        # Check if it's a binary operator
        if self.value in BINARY_OPERATORS:
            left_val = self.left.evaluate_tree_from_node()
            right_val = self.right.evaluate_tree_from_node()
            return BINARY_OPERATORS[self.value](left_val, right_val)
        
        # Check if it's a unary operator
        elif self.value in UNARY_OPERATORS:
            right_val = self.right.evaluate_tree_from_node() # Typically applies to right child
            return UNARY_OPERATORS[self.value](right_val)  # Correct unary application
        
        # Check if it's a variable
        elif self.value in VARIABLES_MAP:
            # return VARIABLES_MAP[self.value]  # Lookup the variable value
            return np.multiply(self.coefficient, VARIABLES_MAP[self.value])  # Lookup the variable value
        
        # Check if it's a numeric constant or coefficient
        elif isinstance(self.value, (int, float)):
            return self.value  # Return as-is for numeric leaf selfs
        
        # If none of the above, it's an error
        else:
            raise ValueError(f"Invalid self value: {self.value}")

    def print_tree_from_node(self):
        print_expr(self)

    def print_tree_values_from_node(self):
        print_expr_values(self)


class Tree:
    def __init__(self, root, depth):
        self.root = root
        self.depth = depth
    
    def __copy__(self):
        return Tree(self.root, self.depth)
    
    def get_nodes(self):
        return self.root.get_nodes_from_node()

    def get_non_leaves_nodes(self):
        return self.root.get_non_leaves_nodes_from_node()
    
    def get_leaves_nodes(self):
        return self.root.get_leaves_nodes_from_node()
    
    def validate_tree(self):
        return self.root.validate_tree_from_node()
    
    def evaluate_tree(self):
        return self.root.evaluate_tree_from_node()
    
    def print_tree(self):
        self.root.print_tree_from_node()

    def print_tree_values(self):
        self.root.print_tree_values_from_node()

    def _update_depths_to_root(self, start_node):
        """Update depths from start_node up to root"""
        current = start_node
        while current:
            current._update_depth()
            current = self._find_parent(self.root, current)
        
    def get_max_depth(self):
        """Get the maximum depth of the tree"""
        return self.root.depth if self.root else 0

    

def random_initial_tree(depth, maxdepth, variables, binary_operators, unary_operators):
    if depth == maxdepth:  # Add a variable until they are all chosen, if yes add a number
        if variables:
            var = random.choice(variables)
            leaf = TreeNode(var)
            leaf.depth = depth + 1
            leaf.coefficient = compute_coefficient(var)
            variables.remove(var)
        else:
            # leaf = TreeNode(compute_coefficient())  # TODO change to compute constant
            leaf = TreeNode(generate_safe_constant())
            leaf.coefficient = 1
        leaf.depth = 1
        return leaf
    
    elif depth == maxdepth - 1: # Add a unary operator
        node = TreeNode(None)
        node.right = random_initial_tree(depth + 1, maxdepth, variables, binary_operators, unary_operators)
        node.left = None
    
        available_unary = [op for op in unary_operators if are_compatible(op, np.multiply(VARIABLES_MAP[node.right.value], node.right.coefficient) if node.right.value in VARIABLES_MAP else node.right.value)]
        available_weights = list(compute_weights_sim(available_unary).values())
        node.value = np.random.choice(available_unary, p=available_weights) # If a choice of a variant unary operator was made, choose a random variant from all the possible ones
        node.depth = depth + 1
        return node
    
    else: # Add a binary operator
        node = TreeNode(None)
        node.left = random_initial_tree(depth + 1, maxdepth, variables, binary_operators, unary_operators)
        node.right = random_initial_tree(depth + 1, maxdepth, variables, binary_operators, unary_operators)
        available_binary = [op for op in binary_operators if are_compatible(op, node.right.evaluate_tree_from_node(), node.left.evaluate_tree_from_node())]
        node.value = random.choice(available_binary) # Choose a random binary operator from all the possible ones
        node._update_depth()
        return node
    
def get_parent(root, target):
    """
    Find the parent of a target node in the tree.
    """
    # The 'is' operator is unaffected by __eq__ and always checks if two variables refer to the same object in memory.
    # https://chatgpt.com/share/678294b0-ed60-8004-932d-40c051582d22
    if not root or root is target:
        return None
    if root.left is target or root.right is target:
        return root
    return get_parent(root.right, target) or get_parent(root.left, target)

def validate_after_replacement(root, replaced_node, unary_operators, binary_operators):
    """
    Validate the tree after replacing a subtree.

    Args:
        root (TreeNode): The root of the tree.
        replaced_node (TreeNode): The node that was replaced.
        unary_operators (list): List of unary operators.
        binary_operators (list): List of binary operators.

    Returns:
        bool: True if the tree is valid, False otherwise.
    """
    current = replaced_node
    while current:
        parent = get_parent(root, current)
        if (parent and parent.value in unary_operators and not are_compatible(parent.value, current.evaluate_tree_from_node())) \
        or (parent and parent.value in binary_operators and not are_compatible(parent.value, current.evaluate_tree_from_node(), parent.left.evaluate_tree_from_node())):
            return False
        current = parent  # Traverse up to the root
    return True
    
def swap_subtrees(source_tree, target_tree):
    """
    Try to swap subtrees from source_tree to target_tree.

    Args:
        source_tree (Tree): The source tree.
        target_tree (Tree): The target tree.

    Returns:
        bool: True if a swap was successful, False otherwise.
    """


    # nodoA = random.choice(lista di nodi di A)
    source_nodes = source_tree.get_nodes()

    while source_nodes:
        source_node = random.choice(source_nodes)
        if try_swap(source_node, target_tree):
            return True
        source_nodes.remove(source_node)
    return False

def try_swap(source_node: TreeNode, target_tree: Tree, filter_leaves_parents=False):
    target_nodes = target_tree.get_non_leaves_nodes()
    if filter_leaves_parents:
        # Consider only nodes whose children are not leaves to avoid adding subtree to leaf in mutation
        target_nodes = [node for node in target_nodes if node.right.right or node.right.left]
    unary_operators = list(UNARY_OPERATORS.keys())
    binary_operators = list(BINARY_OPERATORS.keys())
    while target_nodes:
        # target_node -> parent del nodo in target_tree che verrà sostituito con source_tree
        target_node = random.choice(target_nodes)

        if (target_node.value in unary_operators and are_compatible(target_node.value, source_node.evaluate_tree_from_node())):
            tmp = target_node.right
            target_node.right = source_node
            if validate_after_replacement(target_tree.root, target_node, unary_operators, binary_operators):
                return True
            target_node.right = tmp

        elif (target_node.value in binary_operators):
            choice = random.choice(["right", "left"])
            if choice == "right" and are_compatible(target_node.value, source_node.evaluate_tree_from_node(), target_node.left.evaluate_tree_from_node()):
                tmp = target_node.right
                target_node.right = source_node
                if validate_after_replacement(target_tree.root, target_node, unary_operators, binary_operators):
                    return True
                target_node.right = tmp
            elif choice == "left" and are_compatible(target_node.value, target_node.right.evaluate_tree_from_node(), source_node.evaluate_tree_from_node()):
                tmp = target_node.left
                target_node.left = source_node
                if validate_after_replacement(target_tree.root, target_node, unary_operators, binary_operators):
                    return True
                target_node.left = tmp
        target_nodes.remove(target_node)
    return False

In [992]:
def generate_initial_solution(input_variables=list(VARIABLES_MAP.keys())[:]):
    variables = input_variables[:]
    n_variables = len(variables)
    if n_variables != 0:
        n_leaves = int(2 ** np.ceil(np.log2(n_variables)))
        n_actual_leaves = n_leaves * random.choice([2, 4])
        binary_operators = list(BINARY_OPERATORS.keys())[:]
        unary_operators = list(UNARY_OPERATORS.keys())[:]
        max_depth = np.log2(n_actual_leaves)
    # else :
    #     n_leaves = 1
    #     n_actual_leaves = n_leaves * 2
    #     binary_operators = list(BINARY_OPERATORS.keys())[:]
    #     unary_operators = list(UNARY_OPERATORS.keys())[:]
    #     unary_operators.remove("")
    #     max_depth = np.log2(n_actual_leaves)

    while True:
        root = random_initial_tree(0, max_depth, variables, binary_operators, unary_operators)
        try:
            #root.print_tree_from_node()
            if root.validate_tree_from_node():
                root.evaluate_tree_from_node()
                tree = Tree(root, max_depth)
                return tree
            else:
                print("not valid")
        except:
            pass
        variables = input_variables[:]

## Steps
- We treat a possible solution as a tree. The tree has attribute root, which is the root of the tree of class TreeNode and max_depth.
- Generate random tree
    - we need each variable at least once 
    - each variable has exactly one coefficient chosen as a random float number in the range [?, ?]
    - each variable has exactly one unary operator
    - unary operator is chosen as: 50% chance of "" (i.e. no change to the variable), 50% chance of choosing among all other unary operators
        - check if the unary operator is appliable to the variable ->
            ```
            leaves_map = {}
            for e in leaves:
                available_unary_operators = [op for op in list(UNARY_OPERATORS.keys()) if op.is_applicable(e)]
                chosen_unary_operator = 50% chance of "" (i.e. no change to the variable), 50% chance of choosing among [available_unary_operators]
                leaves_map[e] = [chosen_unary_operator]
            # leaves = [-2, 3]
            # leaves_map = {-2: square, 3: log}
            for e in leaves:
                node = leaves_map[e]
                node.left = null
                node.right = e
                # insert node to tree
            ```
    - number of leaves = nearest power of two greater than keys.length()
    - number of actual leaves = [number of leaves] * 2
    - number of coefficients = [number of leaves] - keys.length()
    - number of binrary operators = total number of nodes in  tree with [number of leaves] leaves - [number of leaves]]
    - validate tree
    - if valid, return tree
    - else, ?
- Example:
    - x.length() = 3
    - number of leaves = 4
    - number of actual leaves = 8
    - number of coefficients = 1
    - number of operands = 3

    ```bash
                    +
            /                  \
            *                    +
        /      \           /        \
      u        1          1          u
    /   \    /   \      /   \       /  \
    nul  *  nul   *    nul    *     nul *
    ```
### EA approach
- Individual is rapresented as a tree and a fitness
    - fitness is a tuple of 2 values: (-mse, right_sign_100)
        - right_sign_100 is the percentage of correct sign predictions
        - mse is the mean squared error
- Classic Genetic Programming
    - Key elements 
    - Representation: tree structures
    - Recombination: exchange of subtrees
    - Mutation: random change in trees
        - subtree mutation -> replace  entire subtree
        - point  mutation -> change single node
        - permutation -> exchange node right with left
        - hoist -> take subtree and make it root
        - expansion -> take random leaf and replace it with a new subtree
        - collapse -> take a subtree and replace it with leaf
    - Population model: generational
    - Parent selection: fitness proportional
    - Survivor selection: deterministic

##next

    - finish implementing mutations
    - find way to reduce tree dimension
    - add weights to opertors
    - add check if fitness doesn't improve , stop early
    - 

##problems
    - overflow
    - initial tree with 4xnodes has invalid values
    - 


In [972]:
# random_tree = generate_initial_solution()
# random_tree.print_tree()
# random_tree2 = generate_initial_solution()
# random_tree2.print_tree()
# if random_tree.validate_tree() and random_tree2.validate_tree():
#     print("valid")
#     # random_tree.print_tree_values()
#     # nodes = random_tree.get_nodes()
#     # for node in nodes:
#     #     print(node.value)

#     # try swap
#     cp1 = copy.deepcopy(random_tree)
#     cp2 = copy.deepcopy(random_tree2)

#     if swap_subtrees(cp1, cp2):
#         print("swapped")
#         cp2.print_tree()
#         cp2.print_tree_values()
#     else:
#         print("not swapped")

    # source_node = random.choice(source_nodes)
    # listB = lista di nodi di B tranne le leaves di B
    # target_nodes = target_tree.get_non_leaves_nodes()
    # available_target_nodes = [node for node in target_nodes if node in source_node.get_nodes()]
    # target_node = random.choice(available_target_nodes)


## EA

In [973]:
@dataclass
class Individual:
    genome: Tree
    fitness: tuple

In [974]:
up = list(UNARY_OPERATORS.keys())
dict_weights = {op : i for op,i in compute_weights_sim(up).items()}
print(dict_weights)
print(sum(dict_weights.values()))
up.remove("sin")
dict_weights = {op : i for op,i in compute_weights_sim(up).items()}
print(dict_weights)
print(sum(dict_weights.values()))

{'': np.float64(0.17827438485659855), 'sin': np.float64(0.12257107827692593), 'cos': np.float64(0.060135028009058455), 'tan': np.float64(0.06158121715886166), 'log': np.float64(0.0), 'arccos': np.float64(0.057759902730593476), 'arcsin': np.float64(0.061586571027254), 'arctan': np.float64(0.17087986128844349), 'sqrt': np.float64(0.0), 'cbrt': np.float64(0.16454195867014046), 'abs': np.float64(0.06082183578409751), 'reciprocal': np.float64(0.06184816219802644)}
0.9999999999999999
{'': np.float64(0.203178149754293), 'cos': np.float64(0.06853549788507851), 'tan': np.float64(0.07018371019492946), 'log': np.float64(0.0), 'arccos': np.float64(0.06582858314855403), 'arcsin': np.float64(0.0701898119636993), 'arctan': np.float64(0.19475065963505478), 'sqrt': np.float64(0.0), 'cbrt': np.float64(0.18752739349760306), 'abs': np.float64(0.06931824821166943), 'reciprocal': np.float64(0.07048794570911851)}
1.0


## fitness


In [993]:
def fitness(sol: Tree):
    y_computed = sol.root.evaluate_tree_from_node()
    right_sign_100 = 100 * np.sum(np.sign(y_computed) == np.sign(y)) / len(y)
    return  -mse(y_computed, y), right_sign_100


def sort_individuals(population, mse_weight=0.6):
    # Normalize both components to [0,1] scale across the population
    mse_scores = np.array([ind.fitness[0] for ind in population])
    sign_scores = np.array([ind.fitness[1] for ind in population])
    
    # Min-max normalization
    mse_norm = (mse_scores - mse_scores.min()) / (mse_scores.max() - mse_scores.min())
    sign_norm = sign_scores / 100  # Already in [0,100] range
    
    # Combine scores with weighted sum
    combined_scores = mse_weight * mse_norm + (1 - mse_weight) * sign_norm
    
    # Sort population based on combined scores (descending order)
    sorted_population = [x for _, x in sorted(zip(combined_scores, population), 
                                            key=lambda pair: pair[0], 
                                            reverse=True)]
    return sorted_population

## crossover and mutations

In [976]:

def xover(population: list[Individual])-> tuple[Tree, tuple]:
    parents = random.choices(population, k=2)

    # reproduce
    c1 = copy.deepcopy(parents[0].genome)
    c2 = copy.deepcopy(parents[1].genome)

    success = swap_subtrees(c1, c2)

    c_fitness = fitness(c2)

    if not success:
        # mutation
        mutation(c2)

    return c2, c_fitness


def subtree_mutation(target_tree: Tree):
    # generate a random source tree to substitute into target_tree

    num_variables = np.random.randint(1, len(VARIABLES_MAP.keys())) if len(VARIABLES_MAP.keys()) > 1 else 1  # take a random number of variables
    variables = random.sample(list(VARIABLES_MAP.keys()), num_variables)

    source_tree_root = generate_initial_solution(variables).root

    return try_swap(source_tree_root, target_tree, True)

def point_mutation(target_tree: Tree):
    nodes = target_tree.get_nodes()
    while nodes:
        node = random.choice(nodes)
        unary_operators = [op for op in  list(UNARY_OPERATORS.keys()) if op != node.value]
        unary_weights = {op: i for op,i in compute_weights_sim(unary_operators).items()}
        binary_operators = [op for op in list(BINARY_OPERATORS.keys()) if op != node.value]
        # mutate unary operator with another one
        if (node.value in UNARY_OPERATORS.keys()):
            while unary_operators:
                tmp = node.value
                node.value = np.random.choice(list(unary_weights.keys()), p=list(unary_weights.values()))
                if are_compatible(node.value, node.right.evaluate_tree_from_node()) and validate_after_replacement(target_tree.root, node, unary_operators, binary_operators):
                    return True
                unary_operators.remove(node.value)
                unary_weights = {op: i for op,i in compute_weights_sim(unary_operators).items()}
                node.value = tmp

        # mutate binary operator with another one
        elif (node.value in BINARY_OPERATORS.keys()):
            while binary_operators:
                tmp = node.value
                node.value = random.choice(binary_operators)
                if are_compatible(node.value, node.right.evaluate_tree_from_node(), node.left.evaluate_tree_from_node()) and validate_after_replacement(target_tree.root, node, binary_operators, binary_operators):
                    return True
                binary_operators.remove(node.value)
                node.value = tmp

        # mutate variable
        elif node.value in VARIABLES_MAP.keys():
            # change the coefficient
            tmp_c = node.coefficient
            tmp_v = node.value
            node.coefficient = compute_coefficient(node.value)
            node.value = random.choice(list(VARIABLES_MAP.keys()))
            parent = get_parent(target_tree.root, node)
            if (parent.value in unary_operators and are_compatible(parent.value, parent.right.evaluate_tree_from_node()) and validate_after_replacement(target_tree.root, parent, unary_operators, binary_operators)) \
            or (parent.value in binary_operators and are_compatible(parent.value, parent.right.evaluate_tree_from_node(), parent.left.evaluate_tree_from_node()) and validate_after_replacement(target_tree.root, parent, unary_operators, binary_operators)):
                return True
            node.coefficient = tmp_c
            node.value = tmp_v
            
        # mutate constant value
        else :
            # change the constant
            tmp = node.value
            # node.value = compute_coefficient() # TODO change to compute constant
            parent = get_parent(target_tree.root, node)
            node.value = generate_constant(parent.value)
            if (parent.value in unary_operators and are_compatible(parent.value, parent.right.evaluate_tree_from_node()) and validate_after_replacement(target_tree.root, parent, unary_operators, binary_operators)) \
            or (parent.value in binary_operators and are_compatible(parent.value, parent.right.evaluate_tree_from_node(), parent.left.evaluate_tree_from_node()) and validate_after_replacement(target_tree.root, parent, unary_operators, binary_operators)):
                return True
            node.value = tmp
        
        nodes.remove(node)
    return False
    
def permutation_mutation(target_tree: Tree):
    available_nodes = [node for node in target_tree.get_non_leaves_nodes() if node.value in BINARY_OPERATORS.keys()]
    binary_operators = list(BINARY_OPERATORS.keys())
    unary_operators = list(UNARY_OPERATORS.keys())
    
    while available_nodes:  
        target_node = random.choice(available_nodes)
        tmpr = target_node.right
        tmpl = target_node.left
        target_node.right = tmpl
        target_node.left = tmpr
        if are_compatible(target_node.value, target_node.right, target_node.left) and validate_after_replacement(target_tree.root, target_node, unary_operators, binary_operators):
            return True
        target_node.right = tmpr
        target_node.left = tmpl
        available_nodes.remove(target_node)
    return False

def expansion_mutation(target_tree: Tree):
    leaves = target_tree.get_leaves_nodes()
    binary_operators = list(BINARY_OPERATORS.keys())
    unary_operators = list(UNARY_OPERATORS.keys())
    num_variables = np.random.randint(1, len(VARIABLES_MAP.keys())) if len(VARIABLES_MAP.keys()) > 1 else 1  # take a random number of variables
    variables = random.sample(list(VARIABLES_MAP.keys()), num_variables)
    
    while leaves:
        target_node = random.choice(leaves)
        source_tree_root = generate_initial_solution(variables).root
      
        parent = get_parent(target_tree.root, target_node)
        if (parent.value in unary_operators and are_compatible(parent.value, source_tree_root.evaluate_tree_from_node())):
            tmp = parent.right
            parent.right = source_tree_root
            if validate_after_replacement(target_tree.root, parent, unary_operators, binary_operators):
                target_tree._update_depths_to_root(parent)
                return True
            parent.right = tmp

        elif (parent.value in binary_operators):
            choice = random.choice(["right", "left"])
            if choice == "right" and are_compatible(parent.value, source_tree_root.evaluate_tree_from_node(), parent.left.evaluate_tree_from_node()):
                tmp = parent.right
                parent.right = source_tree_root
                if validate_after_replacement(target_tree.root, parent, unary_operators, binary_operators):
                    target_tree._update_depths_to_root(parent)
                    return True
                parent.right = tmp
            elif choice == "left" and are_compatible(parent.value, parent.right.evaluate_tree_from_node(), source_tree_root.evaluate_tree_from_node()):
                tmp = parent.left
                parent.left = source_tree_root
                if validate_after_replacement(target_tree.root, parent, unary_operators, binary_operators):
                    target_tree._update_depths_to_root(parent)
                    return True
                parent.left = tmp
        leaves.remove(target_node)
    return False

# substtitue the root with a subtree
def hoist_mutation(target_tree: Tree):
    nodes = target_tree.get_non_leaves_nodes()
    root = target_tree.root

    if len(nodes) == 1:
        return False
    
    #remove root
    nodes.remove(root)

    node = random.choice(nodes)

    target_tree.root = node
    return True

def collapse_mutation(target_tree: Tree):
    available_nodes = target_tree.get_non_leaves_nodes()

    # Consider only nodes whose children are not leaves
    available_nodes = [node for node in available_nodes if node.right.right or node.right.left]
    
    unary_operators = list(UNARY_OPERATORS.keys())
    binary_operators = list(BINARY_OPERATORS.keys())
    
    max_attempts = 10
    while available_nodes:
        parent = random.choice(available_nodes)
        attempts = 0
        while attempts < max_attempts:
            if random.choice([0, 1]): # create a radnom leaf
                random_leaf = get_random_leaf()
            else:
                random_leaf = random.choice(parent.get_leaves_nodes_from_node())

            if (parent.value in unary_operators and are_compatible(parent.value, random_leaf.evaluate_tree_from_node())):
                tmp = parent.right
                parent.right = random_leaf
                if validate_after_replacement(target_tree.root, parent, unary_operators, binary_operators):
                    return True
                parent.right = tmp

            elif (parent.value in binary_operators):
                choice = random.choice(["right", "left"])
                if choice == "right" and are_compatible(parent.value, random_leaf.evaluate_tree_from_node(), parent.left.evaluate_tree_from_node()):
                    tmp = parent.right
                    parent.right = random_leaf
                    if validate_after_replacement(target_tree.root, parent, unary_operators, binary_operators):
                        return True
                    parent.right = tmp
                elif choice == "left" and are_compatible(parent.value, parent.right.evaluate_tree_from_node(), random_leaf.evaluate_tree_from_node()):
                    tmp = parent.left
                    parent.left = random_leaf
                    if validate_after_replacement(target_tree.root, parent, unary_operators, binary_operators):
                        return True
                    parent.left = tmp
            attempts += 1
        available_nodes.remove(parent)
    return False

def get_random_leaf():
    if random.choice([0, 1]): # TODO could choose larger subtree
        random_leaf = TreeNode(random.choice(list(VARIABLES_MAP.keys())))
        random_leaf.coefficient = compute_coefficient(random_leaf.value)
    else:
        random_leaf = TreeNode(generate_safe_constant())
        random_leaf.coefficient = 1
    return random_leaf


MUTATIONS = {
    "subtree": subtree_mutation,
    "point": point_mutation,
    "permutation": permutation_mutation,
    "hoist": hoist_mutation,
    "expansion": expansion_mutation,
    "collapse": collapse_mutation
}

MUTATIONS_WEIGHTS = {
    "subtree": 0.25 ,
    "point": 0.3 ,
    "permutation": 0.05,
    "hoist": 0.2,
    "expansion": 0.1,
    "collapse": 0.1
}


def mutation(genome: Tree):
    available_mutations = list(MUTATIONS.keys())
    available_mutations_weights = {m:p for m,p in MUTATIONS_WEIGHTS.items()}

    w_mutations = list(available_mutations_weights.values())
    while available_mutations:
        mutation = np.random.choice(available_mutations, p=w_mutations)
        # print(mutation)
        if MUTATIONS[mutation](genome):
            return True
        # scale weights 
        to_remove = w_mutations[available_mutations.index(mutation)]
        w_mutations.remove(to_remove)

        # recompute weights
        w_mutations = [w / sum(w_mutations) for w in w_mutations]

        available_mutations.remove(mutation)
    return False




In [990]:
tree = generate_initial_solution()
nodes = tree.get_nodes()
tree.print_tree()
for n in nodes:
    print(f"node : {n.value} depth : {n.depth}")
print("mutate")

expansion_mutation(tree)
tree.print_tree()
for n in nodes:
    print(f"node : {n.value} depth : {n.depth}")
print("mutate")
# leaves = tree.get_leaves_nodes()
# print(f"leaves:")
# for l in leaves:
#     print(l.value)
# leaf = random.choice(leaves)

# parent = get_parent(tree.root, leaf)
# print(f"parent of {leaf.value} is {parent.value}")

(sin (10.015993129077867 X_0 ))* (arctan (0.32265347886198437 X_1 )) = y
node : * depth : 3
node : sin depth : 2
node : X_0 depth : 1
node : arctan depth : 2
node : X_1 depth : 1
mutate
(sin (10.015993129077867 X_0 ))* (arctan ( (32.265347886198434 X_1 ))) = y
node : * depth : 3
node : sin depth : 2
node : X_0 depth : 1
node : arctan depth : 2
node : X_1 depth : 1
mutate


In [979]:
""" test = generate_initial_solution()
test.print_tree()
collapse_mutation(test)
test.print_tree()
print(point_mutation(test))
 test.printtree() """


' test = generate_initial_solution()\ntest.print_tree()\ncollapse_mutation(test)\ntest.print_tree()\nprint(point_mutation(test))\n test.printtree() '

In [946]:
POPULATION_SIZE = 150
OFFSPRING_SIZE = 20
MAX_ITERATIONS = 250

In [ ]:
initial_population = [Individual(generate_initial_solution(), 0) for i in range(POPULATION_SIZE)]
# set the fitness
for ind in initial_population:
    ind.fitness = fitness(ind.genome)

# print(initial_population)

for i in tqdm(range(MAX_ITERATIONS)):
# for i in range(MAX_ITERATIONS):
    if random.random() < 0.6:
        for _ in range(OFFSPRING_SIZE):
            # crossover
            child, c_fitness = xover(initial_population)
    
            initial_population.append(Individual(child, c_fitness))

    # mutation
    else :
        # take a random individual and mutate it
        ind = random.choice(initial_population)
        mutation(ind.genome)
        # recompute fitness
        ind.fitness = fitness(ind.genome)
            
    initial_population = sort_individuals(initial_population)
    
    if i % 100 == 0:
        print(f"iteration {i} {initial_population[0].fitness}")
    # sort population
    # initial_population.sort(key=lambda x: x.fitness[0], reverse=False)
    # remove worst individual
    initial_population = initial_population[:POPULATION_SIZE]

    if i % 100 == 0:
        print(f"iteration {i} {initial_population[0].fitness}")
initial_population.sort(key=lambda x: x.fitness[0], reverse=False)
print("Best individual has formula:")
initial_population[0].genome.print_tree()
# initial_population[0].genome.print_tree_values()
print(f"Best individual has fitness: ({initial_population[0].fitness[0]}, {initial_population[0].fitness[1]})")



  0%|          | 0/250 [00:00<?, ?it/s]

iteration 0 (np.float64(-79.39713933350885), np.float64(99.6))
iteration 0 (np.float64(-79.39713933350885), np.float64(99.6))
iteration 100 (np.float64(-7.7083790918646), np.float64(99.0))
iteration 100 (np.float64(-7.7083790918646), np.float64(99.0))
iteration 200 (np.float64(-7.7083790918646), np.float64(99.0))
iteration 200 (np.float64(-7.7083790918646), np.float64(99.0))
Best individual has formula:
(cbrt (10.015993129077867 X_0 ))- (sin ( (sin (cbrt (cbrt (sin (cbrt (cbrt ((cbrt (cbrt (sin ((cbrt (0.10015993129077867 X_0 ))- (( (sin (arctan (0.9008729903649939 ))))- (arctan (((0.11426274143567189 )+ ((cbrt (((arctan (10.015993129077867 X_0 ))+ ((cbrt (0.10015993129077867 X_0 ))- (( (sin (0.32265347886198437 X_1 )))- (arctan (10.015993129077867 X_0 )))))+ ((sin (1.2452581113818817 ))* (arctan ( (( (0.11426274143567189 ))* ( (0.594966938660472 ))))))))- (( (sin (0.32265347886198437 X_1 )))- (arctan (10.015993129077867 X_0 )))))+ ((sin (1.2452581113818817 ))* (arctan ( (arctan ( (0.1